** This notebook betters the code of cnn_v3.ipynb. However, commetary is only contained in cnn_v3.ipynb, so either of these not to be deleted. This code was implemented on Colab. Train time 1 min per epoch instead of 20 mins locally. Loading data from zip file takes 20 seconds instead of hours. MSE os 0.21 R^2 of 55% compare to dummy R^2 of 0$. See cnn_v3.ipynb **

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWhNGt_efs79gSrxdl7iZrKRXdibz8MS0OtQMEsDbcfEXGhUC4uDQf4
Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/imagedata.zip" -d "/content"

In [103]:
#paperclip video

In [75]:
import numpy as np
import PIL,scipy
import pandas as pd

from PIL import Image
import glob, random,os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn import metrics


import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from keras.applications.vgg16 import VGG16

In [5]:
from tensorflow import keras
keras.__version__

'2.6.0'

In [6]:
#path_targetfile = '/Users/adityahpatel/Desktop/PYTHON PROGRAMS/milestone2_dating_and_beauty/SCUT-FBP5500_v2/train_test_files/All_labels.txt'
path_targetfile = '/content/All_labels.txt'

df_target_all = pd.read_csv(path_targetfile, delimiter=' ', header=None, names=['filename', 'score'])
df_target_all.head(3)

,filename,score
0,CF437.jpg,2.883333
1,AM1384.jpg,2.466667
2,AM1234.jpg,2.150000


In [42]:
df_train, df_test = train_test_split(df_target_all, test_size=0.2, random_state=1, shuffle=True)

In [43]:
df_test.head()

,filename,score
4410,CM345.jpg,2.483333
2256,AF1780.jpg,3.683333
3684,AM974.jpg,2.483333
4706,AF952.jpg,3.266667
5429,CF154.jpg,2.283333


In [9]:
IMAGES_DIR = "/content/Images"

In [10]:
training_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_dataframe(
    directory=IMAGES_DIR, 
    dataframe=df_train,   # contains x_col, y_col targe
    x_col="filename",
    y_col="score",
    target_size=(350,350),
    class_mode='other',
    batch_size=8)

Found 4400 validated image filenames.


In [91]:
cnn = models.Sequential([
    # each filter is a feature detector. Convolving feature detector with original image gives feature map 
    layers.Conv2D(filters=100, kernel_size=(10, 10), activation='relu', input_shape=(350, 350, 3)),
    layers.MaxPooling2D((10, 10)),
    layers.Conv2D(filters=50, kernel_size=(10, 10), activation='relu'),
    layers.MaxPooling2D((10, 10)),
   
    
    # conventional ann
    layers.Flatten(),
    layers.Dense(30, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(1, activation = 'linear')
])

cnn.compile(optimizer='adam',
              loss="mean_squared_error",
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [92]:
mysteps_per_epoch = 4400//8   # Bhattiprolu says this is 'cos batch size is 8

In [93]:
history = cnn.fit(train_generator,
                  steps_per_epoch=mysteps_per_epoch,
                  epochs=30,
                  verbose=1)

Epoch 1/30
550/550 [==============================] - 74s 133ms/step - loss: 0.5159 - root_mean_squared_error: 0.7183
Epoch 2/30
550/550 [==============================] - 74s 134ms/step - loss: 0.3066 - root_mean_squared_error: 0.5537
Epoch 3/30
550/550 [==============================] - 73s 133ms/step - loss: 0.2752 - root_mean_squared_error: 0.5246
Epoch 4/30
550/550 [==============================] - 73s 133ms/step - loss: 0.2252 - root_mean_squared_error: 0.4745
Epoch 5/30
550/550 [==============================] - 73s 133ms/step - loss: 0.2247 - root_mean_squared_error: 0.4740
Epoch 6/30
550/550 [==============================] - 73s 133ms/step - loss: 0.2047 - root_mean_squared_error: 0.4524
Epoch 7/30
550/550 [==============================] - 73s 133ms/step - loss: 0.1790 - root_mean_squared_error: 0.4231
Epoch 8/30
550/550 [==============================] - 73s 133ms/step - loss: 0.1734 - root_mean_squared_error: 0.4164
Epoch 9/30
550/550 [==============================] - 73

In [94]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=IMAGES_DIR,
        x_col="filename",
        batch_size=1,
        shuffle=False,
        target_size=(350, 350),
        class_mode=None)

Found 1100 validated image filenames.


In [95]:
test_generator.reset()
cnn.evaluate(test_generator)

1100/1100 [==============================] - 6s 5ms/step - loss: 0.0000e+00 - root_mean_squared_error: 0.0000e+00


[0.0, 0.0]

In [96]:
cnn.predict(test_generator,verbose=1)

1100/1100 [==============================] - 9s 9ms/step


array([[2.4399161],
       [3.9287348],
       [3.1949563],
       ...,
       [3.5605464],
       [2.5704284],
       [3.3116336]], dtype=float32)

In [97]:
predicted_score = cnn.predict(test_generator,verbose=1)

1100/1100 [==============================] - 9s 9ms/step


In [98]:
df_test.head()

,filename,score
4410,CM345.jpg,2.483333
2256,AF1780.jpg,3.683333
3684,AM974.jpg,2.483333
4706,AF952.jpg,3.266667
5429,CF154.jpg,2.283333


In [99]:
df_report = df_test.copy()
df_report['predicted_score'] = predicted_score

In [100]:
df_report[:20]

,filename,score,predicted_score
4410,CM345.jpg,2.483333,2.439916
2256,AF1780.jpg,3.683333,3.928735
3684,AM974.jpg,2.483333,3.194956
4706,AF952.jpg,3.266667,3.922386
5429,CF154.jpg,2.283333,2.433738
1825,CM732.jpg,4.400000,4.041935
3408,AF1494.jpg,3.700000,2.508137
4306,CF259.jpg,2.933333,3.113216
386,AM1932.jpg,3.000000,3.198035
1751,CM186.jpg,4.116667,3.790813


In [101]:
metrics.r2_score(df_report['score'], df_report['predicted_score'])

0.5518859486109224

In [102]:
metrics.mean_squared_error(df_report['score'], df_report['predicted_score'])

0.20790051483899394